In [8]:
import requests
import pandas as pd
import os
import datetime as dt
from datetime import timedelta
import json
import pandas as pd
import locale
import numpy as np
import matplotlib.pyplot as plt
import pyodbc
from sqlalchemy import create_engine, text
from dateutil.parser import parse
import gender_guesser.detector as gender
import warnings

# Suprimir las advertencias de tipo FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)


# Configurar locale para español (Perú)
locale.setlocale(locale.LC_TIME, 'es_PE.UTF-8')

# Variables PERIODO ABRIL 2024
fecha = '202409'
# Mes a partir del fecha
mes = dt.datetime.strptime(fecha, '%Y%m').strftime('%B').upper()
usuario = 'azaer'
Ruta = rf'C:\Users\{usuario}\Documents\Diego\Reportes'
hoy = dt.datetime.now().strftime('%Y%m%d')

In [9]:
# Definir los parámetros de conexión
server = r'server'
database = 'database'
schema = 'dbo'
username = 'usuario'
password = 'password'
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password};'


# Crear el motor de conexión
engine = create_engine(f'mssql+pyodbc://sa:sanandreas666@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server')

# Crear la conexión
connection = pyodbc.connect(connection_string)

# Definir la consulta SQL
query = text(f"""
Select * From Reportes_Chubb..clientes where aniomes = '{fecha}'
""")

query1 = text(f"""
SELECT * FROM Reportes_Chubb..adicionales where feC_VENTA like '{fecha}%'
""")

# Ejecutar la consulta y leer los datos en un DataFrame de pandas
with engine.connect() as connection:
    df_clientes = pd.read_sql((query), connection)
    df_adicionales = pd.read_sql((query1), connection)


# Reemplazar None (valores nulos) por una cadena vacía
df_clientes = df_clientes.fillna('')
df_adicionales = df_adicionales.fillna('')

def normalize_date(date_str):
    try:
        # Intentar convertir las fechas en formato "YYYY-MM-DD"
        return pd.to_datetime(date_str, format='%Y%m%d', errors='coerce')
    except:
        pass
    
    try:
        # Intentar convertir las fechas en formato "YYYYMMDD"
        return pd.to_datetime(date_str, format='%Y-%m-%d', errors='coerce')
    except:
        return pd.NaT
    
df_adicionales['feC_NACIMIENTO'] = df_adicionales['feC_NACIMIENTO'].apply(normalize_date).dt.strftime('%d/%m/%Y')
df_clientes['fechanacimiento'] = df_clientes['fechanacimiento'].apply(normalize_date).dt.strftime('%d/%m/%Y')

In [10]:
# Reporte de Resultante

df_resultante = df_clientes.copy() 

df_resultante = pd.DataFrame({
    'DNI' : df_resultante['dni'].str[-8:],
    'INTENTOS CONTACTO' :  pd.to_numeric(df_resultante['intentoS_MEDIATEL'], errors='coerce').fillna(0).astype('int64'),
    'FECHA MEJOR GESTIÓN' : pd.to_datetime(df_resultante['fecha'], errors='coerce').dt.strftime('%d/%m/%Y'),
    'HORA MEJOR GESTIÓN' : pd.to_datetime(df_resultante['hora'], errors='coerce').dt.strftime('%H:%M:%S'),
    'CALL' : 'CALLSOUTH PERU',
     'ESTADO' : df_resultante['niveL_3'].apply(lambda x: 'A' if x == 'VOLVER A LLAMAR' else 'T'),
    'MOTIVO' : df_resultante['niveL_3'],
    'NOMBRE DE BBDD' : df_resultante['campana'].map(lambda x: 'RIPLEY PROTECCIÓN ACCIDENTAL' if 'ACCIDENTES_PERSONALES' in x or 'CHUBB_AP_P' in x else
                                                  ('RIPLEY SONRIE SEGURO' if 'SONRIE_SEGURO' in x or 'CHUBB_SONRIE_SEGURO_P' in x else 'RIPLEY CAMINA SEGURO')) + '-' + df_resultante['captación'].str.upper() + '-' + mes,
    'CODIGO DEL EJECUTIVO' : '',
    'NOMBRE DEL EJECUTIVO' : df_resultante['agente'],
    'TipoContrato' : df_resultante['tipoContrato'],
    'Cel 1' : df_resultante['ceL1'],
    'Cel 2' : df_resultante['ceL2']       })

# Filtrar si Motivo es vacio
df_resultante = df_resultante[df_resultante['MOTIVO'] != '']


# Crear una columna que indica cuántas veces se debe duplicar cada fila
df_resultante['Duplicar'] = 1  # Por defecto no duplicar

df_resultante.loc[df_resultante['TipoContrato'].str.contains(r'1', regex=True), 'Duplicar'] = 2
df_resultante.loc[df_resultante['TipoContrato'].str.contains(r'2', regex=True), 'Duplicar'] = 3
df_resultante.loc[df_resultante['TipoContrato'].str.contains(r'3', regex=True), 'Duplicar'] = 4

# Duplicar las filas según el valor en la columna 'Duplicar'
df_resultante = df_resultante.loc[np.repeat(df_resultante.index.values, df_resultante['Duplicar'])]

# Eliminar la columna 'Duplicar'
df_resultante = df_resultante.drop(columns=['Duplicar'])


# Homologar columna Motivo 
df_resultante['MOTIVO'] = df_resultante['MOTIVO'].replace({
    'VENTA': 'Venta',
    'CLIENTE CORTO LLAMADA CON INFO': 'No desea Cliente no escucha propuesta',
    'REVALIDACION': 'No desea Solicita información adicional',
    'ND LO LLAMARON MAS DE UNA VEZ': 'No desea Oferta - No le interesa',
    'ND NO TIENE TARJETA': 'No califica - Canceló tarjeta',
    'NO DESEA - YA LE OFRECIERON': 'No desea Oferta - No le interesa',
    'ND COYUNTURAL': 'No desea No necesita',
    'ND POR COSTO': 'No desea Tema del Cliente Económico',
    'ND NO CONTRATA NADA POR TELF.': 'NO VOLVER A LLAMAR Protección de datos',
    'VOLVER A LLAMAR': 'Agendado a solicitud del titular - Mañana',
    'ND NO BRINDA MOTIVO' : 'No desea Cliente corto llamada',
    'ND NO CONFORME RIPLEY' : 'No desea - insatisfecho con Ripley',
    'CONTACTO CON TERCERO' : 'Agendado por no encontrar al titular -  Mañana',
    'CORTA LLAMADA SIN INFO' : 'Agendado por no encontrar al titular -  Mañana',
    'NUMERO EQUIVOCADO' : 'No corresponde',
    'TITULAR INUBICABLE' : 'Contacto con Terceros día',
    'NO CONTESTA / OCUPADO' : 'No contesta día',
    'GRABADORA' : 'Grabadora',
    'BUZON DE VOZ' : 'Grabadora',
    'FUERA DE SERVICIO / SUSPENDIDO' : 'Fuera de Servicio, malogrado, línea saturada',
    'NO CONTACTADOS' : 'No contesta día',
    'NO CONTESTA' : 'No contesta día',
    'NO CORRESPONDE' :  'No corresponde'
}) 

# Ordenar columna por Fecha mejor gestion y hora mejor gestion
df_resultante = df_resultante.sort_values(by=['FECHA MEJOR GESTIÓN', 'HORA MEJOR GESTIÓN'], ascending=[True, True])

# Calcular el Numero de Ventas en resultante
Ventas_resultantes =len(df_resultante[df_resultante['MOTIVO'] == 'Venta'])

print(f'Ventas en Resultante igual a: {Ventas_resultantes} ')   

# Eliminar columnas innecesarias (TipoContrato, Cel 1, Cel 2)
df_resultante = df_resultante.drop(columns=['TipoContrato', 'Cel 1', 'Cel 2'])


# Guardar el dataframe en un archivo excel
df_resultante.to_excel(f'{Ruta}\\Resultantes\\Reporte_Resultante_{fecha}.xlsx', index=False)


C:\Users\azaer\AppData\Local\Temp\ipykernel_34460\4236202358.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'HORA MEJOR GESTIÓN' : pd.to_datetime(df_resultante['hora'], errors='coerce').dt.strftime('%H:%M:%S'),


Ventas en Resultante igual a: 270 


In [11]:
# Reporte de Validacion de Ventas

df_validacion = df_clientes.copy()

# Filtrar solo las ventas
df_validacion = df_validacion[df_validacion['niveL_3'] == 'VENTA']

df_validacion = pd.DataFrame({
    'DNI' : df_validacion['dni'].str[-8:],
    'FECHA DE VENTA' : pd.to_datetime(df_validacion['fechavta'], errors='coerce').dt.strftime('%d/%m/%Y'),
    'HORA DE VENTA' : pd.to_datetime(df_validacion['horavta'], errors='coerce').dt.strftime('%H:%M:%S'),
    'NOMBRE CLIENTE' : df_validacion['nombre'],
    'NOMBRE EJECUTIVO' : df_validacion['agente'],
    'NOMBRE PRODUCTO' : df_validacion['campana'].map(lambda x: 'RIPLEY PROTECCIÓN ACCIDENTAL' if 'ACCIDENTES_PERSONALES' in x or 'CHUBB_AP_P' in x else
                                                  ('RIPLEY SONRIE SEGURO' if 'SONRIE_SEGURO' in x or 'CHUBB_SONRIE_SEGURO_P' in x else 'RIPLEY CAMINA SEGURO')),
    'PRIMER ESTADO AUDITORIA' : df_validacion['primeR_ESTADO_CALIDAD'],
    'PRIMER MOTIVO AUDITORIA' : df_validacion['primeR_MOTIVO_CALIDAD'],
    'PRIMER FECHA AUDITORIA' : pd.to_datetime(df_validacion['primeR_FECHA_CALIDAD'], errors='coerce').dt.strftime('%d/%m/%Y'),
    'SEGUNDO ESTADO AUDITORIA' : df_validacion['segundO_ESTADO_CALIDAD'],
    'SEGUNDO MOTIVO AUDITORIA' : df_validacion['segundO_MOTIVO_CALIDAD'],
    'SEGUNDO FECHA AUDITORIA' : pd.to_datetime(df_validacion['segundO_FECHA_CALIDAD'], errors='coerce').dt.strftime('%d/%m/%Y'),
    'NOMBRE CALL CENTER' : 'CALLSOUTH PERU',
    'ANIO' : df_validacion['aniomes'].str[:4],
    'MES' : df_validacion['aniomes'].str[4:6],
    'CALIDAD FINAL' : df_validacion['nuevo_Estado_Calidad'],
    'Ventas_Adiciones' : df_validacion['tipoContrato'].map(
    lambda x: 2 if pd.notnull(x) and '1' in x else
              3 if pd.notnull(x) and '2' in x else
              4 if pd.notnull(x) and '3' in x else
              1 if pd.notnull(x) and 'titular' in x.lower() and 'adicional' not in x.lower() else
              0
)})

# Guardar el dataframe en un archivo excel
df_validacion.to_excel(f'{Ruta}\\Validacion\\Reporte_Validacion_{fecha}.xlsx', index=False)

print('Reporte Validación generado')

Reporte Validación generado


C:\Users\azaer\AppData\Local\Temp\ipykernel_34460\3230604148.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'HORA DE VENTA' : pd.to_datetime(df_validacion['horavta'], errors='coerce').dt.strftime('%H:%M:%S'),


In [12]:
# Reporte de Sumarizado a partir de df_cliente

df_sumarizado = df_clientes.copy()

# Consolidar las 3 compañías y emitir informe
company_ids = [1, 2, 3]
company_names = {
    1: 'SONRIE_SEGURO',
    2: 'ACCIDENTES_PERSONALES',
    3: 'CAMINA_SEGURO'
}
df_ocupacion = pd.DataFrame()

# Iterar sobre las compañías Ocupacion
for company in company_ids:
    url = f'https://app.soluziona.pe/API_QA/Peru/CRM/api/Excel_CRM/CRM/Reporte/Conexiones/Excel/{fecha}/{company}'
    
    try:
        response = requests.get(url,timeout=10)
        response.raise_for_status()  # Verificar si hubo un error en la solicitud
        data = response.json()
        company_df = pd.DataFrame(data)
        company_df['campaña'] = company_names[company]  # Añadir la columna 'campaña'

    except:
        # Si ocurre cualquier error, crea un DataFrame vacío
        print(f"Error al conectar con {url}. Creando DataFrame vacío.")
        company_df = pd.DataFrame()

    # Concatenar el DataFrame de la compañía actual al DataFrame principal
    df_ocupacion = pd.concat([df_ocupacion, company_df], ignore_index=True)

#Crear un DataFrame con las columnas necesarias
sumarizado = pd.DataFrame({
    'numero_documento' : df_sumarizado['dni'].str[-8:],
    'dia' : pd.to_datetime(df_sumarizado['fecha'], errors='coerce').dt.strftime('%d'),
    'semana' : pd.to_datetime(df_sumarizado['fecha'], errors='coerce').dt.strftime('%W'),
    'hora' : pd.to_datetime(df_sumarizado['hora'], errors='coerce').dt.strftime('%H'),
    'nivel_1' : df_sumarizado['niveL_1'],
    'nivel_2' : df_sumarizado['niveL_2'],
    'nivel_3' : df_sumarizado['niveL_3'],
    'tipo_captacion' : df_sumarizado['captación'],
    'nombre' : df_sumarizado['agente'],
    'tmo' : pd.to_timedelta(df_sumarizado['tmo'], errors='coerce').fillna('00:00:00').dt.total_seconds().astype(int),
    'cargado' : 1,
    'tramitado' : df_sumarizado['Tramitado'],
    'contacto' : df_sumarizado['Contacto'],
    'No contacto': np.where(df_sumarizado['Tramitado'] == 1,np.where(df_sumarizado['Contacto'] == 1, 0, 1), 0),
    'valido' :  df_sumarizado['Valido'],
    'no valido' : np.where(df_sumarizado['Tramitado'] == 1,np.where(df_sumarizado['Valido'] == 1, 0, 1), 0),
    'ventas' : df_sumarizado['Ventas'],
    'Ventas_Adicionales' : df_sumarizado['tipoContrato'].map(
    lambda x: 2 if pd.notnull(x) and '1' in x else
              3 if pd.notnull(x) and '2' in x else
              4 if pd.notnull(x) and '3' in x else
              1 if pd.notnull(x) and 'titular' in x.lower() and 'adicional' not in x.lower() else
              0),
    'TipoContrato' : df_sumarizado['tipoContrato'],
    'Marcaciones' : '',
    'MarcacionesMediatel' : pd.to_numeric(df_sumarizado['intentoS_MEDIATEL'], errors='coerce').fillna(0).astype('int64'),
    'calidad_Estado' : df_sumarizado['nuevo_Estado_Calidad'],
    'Campaña' : df_sumarizado['campana']})

# Verificar si la ruta existe y si no, crearla
if not os.path.exists(os.path.join(Ruta,'Sumarizados',f"{fecha}")):
    os.makedirs(os.path.join(Ruta,'Sumarizados',f"{fecha}"))

# Guardar el consolidado en la carpeta de destino y guardar df y df_ocupacion en diferentes hojas
ruta_consolidado = os.path.join(Ruta,'Sumarizados',f"{fecha}", f"Sumarizado_{fecha}.xlsx")
with pd.ExcelWriter(ruta_consolidado) as writer:
    sumarizado.to_excel(writer, sheet_name='Sumarizado', index=False)
    df_ocupacion.to_excel(writer, sheet_name='Ocupacion', index=False)


Error al conectar con https://app.soluziona.pe/API_QA/Peru/CRM/api/Excel_CRM/CRM/Reporte/Conexiones/Excel/202409/1. Creando DataFrame vacío.
Error al conectar con https://app.soluziona.pe/API_QA/Peru/CRM/api/Excel_CRM/CRM/Reporte/Conexiones/Excel/202409/2. Creando DataFrame vacío.
Error al conectar con https://app.soluziona.pe/API_QA/Peru/CRM/api/Excel_CRM/CRM/Reporte/Conexiones/Excel/202409/3. Creando DataFrame vacío.


C:\Users\azaer\AppData\Local\Temp\ipykernel_34460\2964783206.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'hora' : pd.to_datetime(df_sumarizado['hora'], errors='coerce').dt.strftime('%H'),


In [13]:
df_trama = df_clientes.copy()

def separar_nombre(nombre_completo):
    # Dividir el nombre completo en partes usando los espacios
    partes = nombre_completo.split()

    # Inicializar las variables
    apellido_paterno = ''
    apellido_materno = ''
    nombre1 = ''
    nombre2 = ''

    # Asignar valores basados en el número de partes
    if len(partes) > 4:
        # Considerar el caso de más de 4 partes
        apellido_materno = partes[-1]
        apellido_paterno = partes[-2]
        nombre1 = partes[0]
        nombre2 = ' '.join(partes[1:-2])
    elif len(partes) == 4:
        nombre1, nombre2, apellido_paterno, apellido_materno = partes
    elif len(partes) == 3:
        nombre1, apellido_paterno, apellido_materno = partes
    elif len(partes) == 2:
        nombre1, apellido_paterno = partes
    elif len(partes) == 1:
        nombre1 = partes[0]

    return apellido_paterno, apellido_materno, nombre1, nombre2

mes = dt.datetime.strptime(fecha, '%Y%m').strftime('%B')

nombres_separados = df_trama['nombre'].apply(separar_nombre).apply(pd.Series)
nombres_separados.columns = ['apellido_paterno', 'apellido_materno', 'nombre1', 'nombre2']
df_trama = pd.concat([df_trama, nombres_separados], axis=1)



#Filtrar solo las ventas en estado S y NS en columna nuevo_Estado_Calidad
df_trama = df_trama[(df_trama['nuevo_Estado_Calidad'] == 'S') | (df_trama['nuevo_Estado_Calidad'] == 'NS')]


#Importar Tabla de Ubicacion para reemplazar códigos
archivo_codigos = rf'C:\Users\{usuario}\Documents\Diego\Reportes\TRAMAS\Tabla de Códigos Trama.xlsx'
df_codigos = pd.read_excel(archivo_codigos, sheet_name='Tabla 2 Datos vivienda' , dtype=str)

#Reemplazar en df_trama si departamento, provincia y distrito es piura, piura, 0 por piura, piura, piura , solo si se cumple la condición
df_trama.loc[(df_trama['departamento'] == 'piura') & (df_trama['provincia'] == 'piura') & (df_trama['distrito'] == '0'), 'distrito'] = 'piura'

# Concatenar en la tabla de clientes
df_trama['concat'] = df_trama['departamento']+ df_trama['provincia']+ df_trama['distrito']

df_trama = df_trama.merge(df_codigos[['J', 'UBIGEO']], left_on='concat', right_on='J', how='left')

df_trama['DEPARTAMENTO_ID'] = df_trama['UBIGEO'].str[:2]
df_trama['PROVINCIA_ID'] = df_trama['UBIGEO'].str[2:4]
df_trama['DISTRITO_ID'] = df_trama['UBIGEO'].str[4:]

mapeo_avanzado = {
    'CONYUGE': {'parentescO_ID': '38', 'tipO_ASEG': 'AC'},
    'HIJO': {'parentescO_ID': '39', 'tipO_ASEG': 'AH'},
    'HERMANO': {'parentescO_ID': '40', 'tipO_ASEG': 'AO'},
    'PADRE': {'parentescO_ID': '42', 'tipO_ASEG': 'AO'},
    'MADRE': {'parentescO_ID': '43', 'tipO_ASEG': 'AO'},
    'OTRO FAMILIAR': {'parentescO_ID': '44', 'tipO_ASEG': 'AO'},
    'OTRA RELACION': {'parentescO_ID': '45', 'tipO_ASEG': 'AO'},
    'HIJA': {'parentescO_ID': '1347', 'tipO_ASEG': 'AH'},
    'HERMANA': {'parentescO_ID': '1348', 'tipO_ASEG': 'AO'},
    'SOBRINO': {'parentescO_ID': '1349', 'tipO_ASEG': 'AO'},
    'SOBRINA': {'parentescO_ID': '1352', 'tipO_ASEG': 'AO'}
}

# Crear un detector de género
d = gender.Detector()

def asignar_prima(tipo_contrato):
    if pd.notnull(tipo_contrato):
        tipo_contrato = tipo_contrato.lower()
        if '1' in tipo_contrato:
            return '44.9'
        elif '2' in tipo_contrato or '3' in tipo_contrato:
            return '59.9'
        elif 'titular' in tipo_contrato and 'adicional' not in tipo_contrato:
            return '29.9'
    return '0'

def asignar_producto(campana):
    if pd.notnull(campana):
        if 'ACCIDENTES_PERSONALES' in campana or 'CHUBB_AP_P' in campana:
            return 'RIPLEY PROTECCIÓN ACCIDENTAL'
        elif 'SONRIE_SEGURO' in campana or 'CHUBB_SONRIE_SEGURO_P' in campana:
            return 'RIPLEY SONRIE SEGURO'
    return 'RIPLEY CAMINA SEGURO'

def asignar_producto_id(campana):
    if pd.notnull(campana):
        if 'ACCIDENTES_PERSONALES' in campana or 'CHUBB_AP_P' in campana:
            return '95'
        elif 'SONRIE_SEGURO' in campana or 'CHUBB_SONRIE_SEGURO_P' in campana:
            return '94'
    return '96'

def asignar_plan_id(tipo_contrato, campana):
    if pd.notnull(campana):
        campana = campana.upper()
        tipo_contrato = tipo_contrato.lower() if pd.notnull(tipo_contrato) else ""

        if 'CAMINA' in campana:
            return '216'
        elif 'AP' in campana or 'ACCIDENTES_PERSONALES' in campana:
            return '215'
        elif 'SONRIE' in campana:
            if 'titular' in tipo_contrato:
                if '1' in tipo_contrato:
                    return '223'
                elif '2' in tipo_contrato:
                    return '228'
                elif '3' in tipo_contrato:
                    return '224'
                else:
                    return '214'
        elif 'PROTECCIÓN ACCIDENTAL' in campana:
            return '215'
    return 'ERROR'

def normalize_date(date_str):
    try:
        # Intentar convertir las fechas en formato "YYYY-MM-DD"
        return pd.to_datetime(date_str, format='%Y%m%d', errors='coerce')
    except:
        pass
    
    try:
        # Intentar convertir las fechas en formato "YYYYMMDD"
        return pd.to_datetime(date_str, format='%Y-%m-%d', errors='coerce')
    except:
        return pd.NaT
    
def mapear_parentesco_id(descripcion):
    descripcion = descripcion.strip().upper()
    return mapeo_avanzado.get(descripcion, {'parentescO_ID': '45'})['parentescO_ID']

def mapear_tipo_aseg(descripcion):
    descripcion = descripcion.strip().upper()
    return mapeo_avanzado.get(descripcion, {'tipO_ASEG': 'AO'})['tipO_ASEG']

def get_gender(nombre):
    return d.get_gender(nombre)

def generar_observaciones(grupo):
    observaciones = []
    
    # Condiciones generales
    if 'AT' not in grupo['TIPO_ASEG'].values:
        observaciones.append("No hay Titular en el Grupo Familiar")
    
    titular = grupo[grupo['TIPO_ASEG'] == 'AT'].iloc[0] if 'AT' in grupo['TIPO_ASEG'].values else None
    
    # Condiciones para un solo miembro en el grupo
    if len(grupo) == 1:
        if titular is not None:
            if titular['PRIMA'] != '29.9':
                observaciones.append("Prima no corresponde")
            if titular['PRODUCTO'] == 'RIPLEY SONRIE SEGURO' and titular['PRODUCTO_ID'] != '94':
                observaciones.append("Producto ID no corresponde")
    else:
        # Más de un miembro en el grupo
        if titular is not None:
            if titular['PRODUCTO'] != 'RIPLEY SONRIE SEGURO':
                observaciones.append("Más de 1 Titular para plan que no corresponde")
        
        # Condiciones para dos miembros en el grupo
        if len(grupo) == 2: 
            if any(grupo['PLAN_ID'] != '223'):
                observaciones.append("Error plan ID")

        # Condiciones para tres o cuatro miembros en el grupo
        if len(grupo) == 3:
            if any(grupo['PLAN_ID'] != '228'):
                observaciones.append("Error plan ID")
        
        # Condiciones para tres o cuatro miembros en el grupo
        if len(grupo) == 4:
            if any(grupo['PLAN_ID'] != '224'):
                observaciones.append("Error plan ID")
        if len(grupo) > 4 :
            observaciones.append("Más de 4 miembros en el grupo")

        # Condición para más de un miembro y producto ID diferente de 94
        if any(grupo['PRODUCTO_ID'] != '94'):
            observaciones.append("Error producto ID")
    
    # Errores de ubicación
    if any(grupo['DEPARTAMENTO'] == '00'):
        observaciones.append("Error en departamento")
    if any(grupo['PROVINCIA'] == '00'):
        observaciones.append("Error en provincia")
    if any(grupo['DISTRITO'] == '00'):
        observaciones.append("Error en distrito")
    
    # Error de longitud del DNI
    if any(grupo['NRO_DOC'].str.len() != 8) or any(~grupo['NRO_DOC'].str.isnumeric()):
        observaciones.append("Error longitud DNI")
    
    return ", ".join(observaciones)


# aplicar las funciones 

df_trama['PRIMA'] = df_trama['tipoContrato'].apply(asignar_prima)
df_trama['PRODUCTO'] = df_trama['campana'].apply(asignar_producto)
df_trama['PRODUCT_ID'] = df_trama['campana'].apply(asignar_producto_id)
df_trama['PLAN_ID'] = df_trama.apply(lambda x: asignar_plan_id(x['tipoContrato'], x['campana']), axis=1)

#Filtrar DNIS que no estan en el archivo total
archivo_total = rf'C:\Users\{usuario}\Documents\Diego\Reportes\TRAMAS\Acumulado Traza Total.xlsx'
df_total = pd.read_excel(archivo_total, sheet_name=f'{mes}' , dtype=str)
df_trama = df_trama[~df_trama['dni'].isin(df_total['NRO_DOC'])]

#Obtener el último código utilizado
ultimo_codigo = int(df_total['NRO_UNICO_CARGA'].max())
print("Último código utilizado:", ultimo_codigo)

#Filtrar Adicionales que tengan el mismo cli_id de df_trama
df_adicionales = df_adicionales[df_adicionales['clI_ID'].isin(df_trama['id'])]

#Ordenar df_trama por fecha de venta , ID 
df_trama = df_trama.sort_values(by=['fechavta', 'id'], ascending=[True, False])

# Calcular el rango para NRO_UNICO_CARGA
nro_unico_carga_range = range(ultimo_codigo + 1, ultimo_codigo + 1 + len(df_trama))


#Formatear columnas y ordenarlas
trama = pd.DataFrame({
    'ITEMS': '',
    'NRO_UNICO_CARGA' : list(nro_unico_carga_range),
    'FEC_VENTA': pd.to_datetime(df_trama['fechavta'], errors='coerce').dt.strftime('%d/%m/%Y'),
    'TIPO_ASEG' : 'AT',
    'PARENTESCO_ID': '125',
    'TIPO_DOC_ID' : '317',
    'NRO_DOC' : df_trama['dni'].astype(str).str.zfill(8), 
    'APE_PATERNO' : df_trama['apellido_paterno'].str.strip().str.upper(),
    'APE_MATERNO' : df_trama['apellido_materno'].str.strip().str.upper(),
    'PRIMER_NOMBRE' : df_trama['nombre1'].str.strip().str.upper(),
    'SEGUNDO_NOMBRE' : df_trama['nombre2'].str.strip().str.upper(),
    'FEC_NACIMIENTO' : df_trama['fechanacimiento'],
    'SEXO_ID' : df_trama['sexo'].map({'m': '120', 'f': '121'}),
    'ESTADO_CIVIL_ID' : '3',
    'EMAIL' : df_trama['email'].str.strip(),
    'TELEFONO_MOVIL' : df_trama['ceL1'].astype(str),
    'PARTICIPACION' : '',
    'PROFESION_ID' : '228',
    'ACTIVIDAD_ID' : '507',
    'CATEGORIA_ID' : '1899',
    'DEPARTAMENTO': df_trama['DEPARTAMENTO_ID'],
    'PROVINCIA': df_trama['PROVINCIA_ID'],
    'DISTRITO': df_trama['DISTRITO_ID'],
    'DIRECCION': df_trama['direccion'].str.strip(),
    'PRIMA' : df_trama['PRIMA'],
    'PRODUCTO_ID' : df_trama['PRODUCT_ID'],
    'PRODUCTO' : df_trama['PRODUCTO'],
    'PLAN_ID' : df_trama['PLAN_ID'],
    'MONEDA_ID' : '545',
    'FRECUENCIA_PAGO_ID' : '30',
    'FORMA_PAGO_ID' : '192',
    'TIPO_TARJETA_ID' : '',
    'NRO_TARJETA' : '9999999999999999',
    'FEC_EXP_TARJETA' : '01/2030',
    'NRO_CUOTAS'   : '1',
    'COD_VENDEDOR' : '99999',
    'SUCURSAL_ID' : '38',
    'CANAL_VENTA_ID' : '7',
    'DPS' : '', 
    'FLG_POLIZA_ELECTRONICA' : 'S',
    'OBSERVACION' : '',
    'CODIGO_CLIENTE' : df_trama['id'],
    'TipoPlan': df_trama['tipoContrato']})

trama_adicionales = pd.DataFrame({
    'ITEMS': '',
    'NRO_UNICO_CARGA' : '',
    'TIPO_ASEG' : df_adicionales['parentescO_ID'].str.strip().apply(mapear_tipo_aseg),
    'PARENTESCO_ID': df_adicionales['parentescO_ID'].str.strip().apply(mapear_parentesco_id),
    'TIPO_DOC_ID' : '317',
    'NRO_DOC' : df_adicionales['nrO_DOC'].astype(str).str.zfill(8), 
    'APE_PATERNO' : df_adicionales['apE_PATERNO'].str.strip().str.upper(),
    'APE_MATERNO' : df_adicionales['apE_MATERNO'].str.strip().str.upper(),
    'PRIMER_NOMBRE' : df_adicionales['clI_ANOMBRE1'].str.strip().str.upper(),
    'SEGUNDO_NOMBRE' : df_adicionales['clI_ANOMBRE2'].str.strip().str.upper(),
    'FEC_NACIMIENTO' : df_adicionales['feC_NACIMIENTO'],
    'SEXO_ID' : df_adicionales['clI_ANOMBRE1'].str.strip().str.title().apply(get_gender).map(lambda x: '121' if 'female' in x else ('120' if 'male' in x else '')),
    'ESTADO_CIVIL_ID' : '3',
    'EMAIL' : '',
    'TELEFONO_MOVIL' : '',
    'PARTICIPACION' : '',
    'PROFESION_ID' : '228',
    'ACTIVIDAD_ID' : '507',
    'CATEGORIA_ID' : '1899',
    'DEPARTAMENTO': '',
    'PROVINCIA': '',
    'DISTRITO': '',
    'DIRECCION': '',
    'PRIMA' : '',
    'PRODUCTO_ID' : '',
    'PRODUCTO' : '',
    'PLAN_ID' : '',
    'MONEDA_ID' : '545',
    'FRECUENCIA_PAGO_ID' : '30',
    'FORMA_PAGO_ID' : '192',
    'TIPO_TARJETA_ID' : '',
    'NRO_TARJETA' : '9999999999999999',
    'FEC_EXP_TARJETA' : '01/2030',
    'NRO_CUOTAS'   : '1',
    'COD_VENDEDOR' : '99999',
    'SUCURSAL_ID' : '38',
    'CANAL_VENTA_ID' : '7',
    'DPS' : '', 
    'FLG_POLIZA_ELECTRONICA' : 'S',
    'OBSERVACION' : '',
    'CODIGO_CLIENTE' : df_adicionales['clI_ID'],
    'TipoPlan': 'Adicional'})

#Cruzar trama_adicionales con df_trama para obtener FechaVenta
trama_adicionales = trama_adicionales.merge(trama[['CODIGO_CLIENTE', 'FEC_VENTA']], left_on='CODIGO_CLIENTE', right_on='CODIGO_CLIENTE', how='left')

# Combinar las tablas
trama_completa = pd.concat([trama, trama_adicionales], ignore_index=True)

# Añadir columna temporal para ordenamiento de TIPO_ASEG
trama_completa['Orden_TIPO_ASEG'] = trama_completa['TIPO_ASEG'].map({'AT': 1, 'AH': 2})

# Reordenar columnas por Fecha de Venta, CODIGO_CLIENTE y Tipo Asegurado
trama_completa = trama_completa.sort_values(by=['CODIGO_CLIENTE','FEC_VENTA', 'Orden_TIPO_ASEG'], ascending=[True, True, True]).drop(columns=['Orden_TIPO_ASEG'])

# Creamos una columna que marque donde comienza un nuevo grupo basado en 'TIPO_ASEG' siendo 'AT'
trama_completa['grupo'] = (trama_completa['TIPO_ASEG'] == 'AT').cumsum()

# Aplicamos el ffill solo dentro de cada grupo
trama_completa = trama_completa.groupby(['CODIGO_CLIENTE', 'grupo'], group_keys=False).apply(lambda g: g.replace('', np.nan).ffill())

# Eliminar la columna auxiliar 'grupo' si ya no es necesaria
trama_completa = trama_completa.drop(columns=['grupo'])

#Asignar indice a columna ITEMS en formato de texto en el orden actual
trama_completa['ITEMS'] = range(1, len(trama_completa) + 1)


# Aplicar la función a cada grupo y generar las observaciones
observaciones_por_grupo = trama_completa.groupby('NRO_UNICO_CARGA').apply(generar_observaciones).reset_index()
observaciones_por_grupo.columns = ['NRO_UNICO_CARGA', 'ERRORES']


# Merge para asignar las observaciones al DataFrame original
trama_completa = trama_completa.merge(observaciones_por_grupo, on='NRO_UNICO_CARGA', how='left')



# Guardar el dataframe en un archivo excel CM_TLMK_V_ + hoy + .xlsx y en una hoja llamada 'Trama'
trama_completa.to_excel(f'{Ruta}\\Tramas\\CM_TLMK_1_V_{hoy}.xlsx', sheet_name='Trama', index=False)



Último código utilizado: 20868


C:\Users\azaer\AppData\Local\Temp\ipykernel_34460\2635938460.py:340: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  trama_completa = trama_completa.groupby(['CODIGO_CLIENTE', 'grupo'], group_keys=False).apply(lambda g: g.replace('', np.nan).ffill())
C:\Users\azaer\AppData\Local\Temp\ipykernel_34460\2635938460.py:350: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  observaciones_por_grupo = trama_completa.group

In [14]:
print(f'Proceso terminado a las ', dt.datetime.now().strftime('%H:%M:%S'))
print(f'Ventas en Resultante igual a: {Ventas_resultantes}')
print(f'Ventas en Validacion igual a: {df_validacion["Ventas_Adiciones"].sum()}')
print(f'Ventas en Sumarizado igual a: {sumarizado["Ventas_Adicionales"].sum()}')


Proceso terminado a las  20:30:35
Ventas en Resultante igual a: 270
Ventas en Validacion igual a: 260
Ventas en Sumarizado igual a: 260
